In [1]:
# install libs
!pip install transformers
!pip install torch --upgrade
!pip install gradio
!pip install beautifulsoup4
!pip install ntlk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.0 MB/s eta 0:00:

In [2]:
# imports
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import gradio as gr
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def pesquisa(product):

    # Makes a request in the free market research tool and generates the html of the page with what was typed by the user
    url = f'https://lista.mercadolivre.com.br/{product}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # elects the items suggested by the free market, based on the research
    produtos = soup.select('.ui-search-layout__item')
    comentarios = []

    # This loop goes into each recommended product and goes into that product's link
    for produto in produtos:
        link_produto = produto.select_one('.ui-search-link')['href']
        response = requests.get(link_produto)
        soup_produto = BeautifulSoup(response.text, 'html.parser')

        # Enters the html class responsible for the list of comments
        avaliacoes = soup_produto.find_all('article', class_='ui-review-capability-comments__comment')

        # Enters the html class responsible for each of the comments, storing them in the comments list
        for avaliacao in avaliacoes:
            comment_text = avaliacao.find('p', {'class': 'ui-review-capability-comments__comment__content'}).text.strip()
            comentarios.append(comment_text)

    return comentarios



In [4]:
from transformers import pipeline
# It call the model
sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


In [5]:

# A function to adaptation about response
def adaptation(response):

    if response == '1 star':
      return 1
    elif response == '2 stars':
      return 2
    elif response == '3 stars':
      return 3
    elif response == '4 stars':
      return 4
    elif response == '5 stars':
      return 5


# Average of stars forms the emojis
def estrelinhas(media):
  estrela_total = []

  while media >=1:
    estrela_total.append('★') # it's equals 1
    media-=1

  if media<1 and media > 0.49:
    estrela_total.append('✫') # it's equals 0.5

  while len(estrela_total)<5:
      estrela_total.append('✰')# it's equals 0


  print(estrela_total)
  # All stars
  estrela_total = estrela_total[0] + estrela_total[1] + estrela_total[2] + estrela_total[3] + estrela_total[4]
  return(estrela_total)

# A fuction to creat a dataframe with comments
def create_df(comentarios):
    sentimentos = []

    for i in comentarios:
        if '' in comentarios: # no need for empty comments
          comentarios.remove('')
        try:
            result = sentiment_analyzer(i)
            response = result[0]['label']

            sentimentos.append(adaptation(response))

        except:
            pass
    # dataframe with two columns Comments and Stars
    df = pd.DataFrame(list(zip(comentarios, sentimentos)), columns=['Comentario', 'Estrelas'])
    return df

# To create a bar graph
def grafico_estrelas(df):
    # Count the stars and their use
    contagem = df['Estrelas'].value_counts()

    # Plot
    plt.bar(contagem.index, contagem.values,color='#cccc00')

    # Set chart labels and title
    plt.xlabel('Estrelas')
    plt.ylabel('Contagem')
    plt.title('Contagem dos tipos de estrela')

    nome_arquivo = 'grafico_estrelas.png'
    plt.savefig(nome_arquivo)
    return nome_arquivo


def nuvem_p(df, product):
  texto = ' '.join(df['Comentario'])
  stop_words = set(stopwords.words('portuguese'))
  product = product.lower()
  product = word_tokenize(product)

  palavras_excluir = ['produto']
  palavras_excluir = palavras_excluir + product

  tokens = word_tokenize(texto)
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words and  word.lower() not in palavras_excluir]


  texto_filtrado = ' '.join(filtered_tokens)

  wordcloud = WordCloud().generate(texto_filtrado)
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.title('Nuvem de palavras')
  plt.axis('off')
  plt.show()

  nome_arquivo2 = 'nuvem_palavras.png'
  plt.savefig(nome_arquivo2)
  return nome_arquivo2



In [6]:
# The prime function
def function (product):
      comentarios = pesquisa(product)
      df = create_df(comentarios)
      result_list = df.to_dict()

      media = df['Estrelas'].mean()
      media = round(media, 2)

      comentarios_text = '\n \n'.join(comentarios)

      grafico_estrelas(df)
      fig1 = grafico_estrelas(df)
      fig2 = nuvem_p(df, product)

      return [estrelinhas(media),comentarios_text, fig1, fig2]

# Interface
iface = gr.Interface(
    fn = function,
    inputs = "text",
    outputs=[
        gr.Textbox(label="Média das estrelas"),
        gr.Textbox(label="Comentários"),
        gr.Image(label="Gráfico de Barras"),
        gr.Image(label="Nuvem de palavras")
            ],
    title = "Pesquisa",

    share = True,
    debug = True,

)

iface.launch()

<ipython-input-6-7e4c212ea317>:19: GradioUnusedKwargWarning: You have unused kwarg parameters in Interface, please remove them: {'share': True, 'debug': True}
  iface = gr.Interface(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>